# ⚡ Intermediaire | ⏱ 45 min | 🔑 Concepts : _, __, @property, encapsulation Pythonic

# Encapsulation et Visibilité en Python

## Objectifs

À la fin de ce notebook, vous serez capable de :
- Utiliser les conventions `_` et `__` pour la visibilité des attributs
- Comprendre le name mangling avec `__`
- Maîtriser les `@property` pour des getters/setters Pythonic
- Utiliser `@attribut.setter` et `@attribut.deleter`
- Implémenter la validation avec `@property`
- Créer des attributs en lecture seule
- Optimiser la mémoire avec `__slots__`

## Prérequis

- Connaissance des classes et de l'instanciation
- Compréhension des attributs d'instance et de classe

## 1. Convention `_` : attributs et méthodes "protégés"

En Python, un underscore simple `_` au début d'un nom indique qu'un attribut ou une méthode est **protégé** par convention.

⚠️ C'est une **convention**, pas une restriction technique. L'accès est toujours possible.

### Signification
- "Ne devrait pas être utilisé directement en dehors de la classe"
- "API interne, peut changer sans préavis"
- Respecté par les IDE (pas d'autocomplétion par défaut)

In [ ]:
class CompteBancaire:
    def __init__(self, titulaire, solde_initial):
        self.titulaire = titulaire  # Public
        self._solde = solde_initial  # Protégé par convention
        self._historique = []  # Protégé par convention
    
    def deposer(self, montant):
        """Méthode publique"""
        if montant > 0:
            self._solde += montant
            self._ajouter_historique("Dépôt", montant)
    
    def _ajouter_historique(self, operation, montant):
        """Méthode protégée (interne)"""
        self._historique.append({"op": operation, "montant": montant})
    
    def get_solde(self):
        return self._solde

compte = CompteBancaire("Alice", 1000)

# ✅ Utilisation normale (API publique)
compte.deposer(500)
print(f"Solde : {compte.get_solde()}€")

# ⚠️ Techniquement possible mais déconseillé
print(f"Accès direct à _solde : {compte._solde}€")
print(f"Historique : {compte._historique}")

## 2. Name mangling `__` : pseudo-privé

Deux underscores `__` au début d'un nom déclenchent le **name mangling** : Python renomme automatiquement l'attribut pour le rendre plus difficile d'accès.

### Transformation
`__attribut` devient `_NomClasse__attribut`

⚠️ Ce n'est **pas vraiment privé** ! C'est pour éviter les collisions de noms dans l'héritage.

In [ ]:
class Voiture:
    def __init__(self, marque, prix):
        self.marque = marque  # Public
        self._kilometrage = 0  # Protégé
        self.__prix_achat = prix  # Pseudo-privé (name mangling)
    
    def get_prix(self):
        return self.__prix_achat
    
    def appliquer_remise(self, pourcentage):
        self.__prix_achat *= (1 - pourcentage / 100)

voiture = Voiture("Renault", 25000)

# ✅ Accès via méthode publique
print(f"Prix : {voiture.get_prix()}€")

# ❌ Erreur : attribut introuvable
try:
    print(voiture.__prix_achat)
except AttributeError as e:
    print(f"Erreur : {e}")

# ⚠️ Mais on peut quand même y accéder avec le nom mangé
print(f"\nAccès via name mangling : {voiture._Voiture__prix_achat}€")

# Voir tous les attributs
print(f"\nAttributs : {[attr for attr in dir(voiture) if not attr.startswith('__')]}")

### Pourquoi le name mangling ?

Le name mangling est conçu pour éviter les collisions de noms dans l'héritage, pas pour la sécurité.

In [ ]:
class Parent:
    def __init__(self):
        self.__secret = "Secret du parent"  # Name mangling
    
    def afficher(self):
        print(f"Parent : {self.__secret}")

class Enfant(Parent):
    def __init__(self):
        super().__init__()
        self.__secret = "Secret de l'enfant"  # Différent nom mangé
    
    def afficher_enfant(self):
        print(f"Enfant : {self.__secret}")

e = Enfant()
e.afficher()  # Accède à _Parent__secret
e.afficher_enfant()  # Accède à _Enfant__secret

# Les deux existent en même temps sans conflit
print(f"\nSecret parent : {e._Parent__secret}")
print(f"Secret enfant : {e._Enfant__secret}")

## 3. `@property` : getters Pythonic

En Python, on préfère utiliser des **propriétés** plutôt que des getters/setters explicites (comme en Java).

`@property` transforme une méthode en attribut (lecture seule par défaut).

In [ ]:
class Personne:
    def __init__(self, prenom, nom):
        self.prenom = prenom
        self.nom = nom
    
    @property
    def nom_complet(self):
        """Propriété calculée dynamiquement"""
        return f"{self.prenom} {self.nom}"
    
    @property
    def initiales(self):
        return f"{self.prenom[0]}.{self.nom[0]}."

p = Personne("Marie", "Curie")

# ✅ Accès comme un attribut (pas de parenthèses)
print(f"Nom complet : {p.nom_complet}")
print(f"Initiales : {p.initiales}")

# Modification des attributs de base
p.prenom = "Pierre"
print(f"Nouveau nom complet : {p.nom_complet}")  # Recalculé automatiquement

# ❌ Erreur : pas de setter défini
try:
    p.nom_complet = "Albert Einstein"
except AttributeError as e:
    print(f"\nErreur : {e}")

### Comparaison : getters Java-style vs @property

In [ ]:
# ❌ Style Java (pas Pythonic)
class RectangleJava:
    def __init__(self, largeur, hauteur):
        self._largeur = largeur
        self._hauteur = hauteur
    
    def get_largeur(self):
        return self._largeur
    
    def set_largeur(self, valeur):
        self._largeur = valeur
    
    def get_aire(self):
        return self._largeur * self._hauteur

# ✅ Style Python (Pythonic)
class RectanglePython:
    def __init__(self, largeur, hauteur):
        self.largeur = largeur  # Public directement
        self.hauteur = hauteur
    
    @property
    def aire(self):
        return self.largeur * self.hauteur

# Comparaison d'utilisation
r1 = RectangleJava(10, 5)
print(f"Java-style : {r1.get_aire()}")  # Appel de méthode

r2 = RectanglePython(10, 5)
print(f"Python-style : {r2.aire}")  # Accès comme un attribut

## 4. `@attribut.setter` : setters Pythonic

Pour rendre une propriété modifiable, on définit un **setter** avec `@attribut.setter`.

In [ ]:
class Temperature:
    def __init__(self, celsius=0):
        self._celsius = celsius
    
    @property
    def celsius(self):
        """Getter pour celsius"""
        return self._celsius
    
    @celsius.setter
    def celsius(self, valeur):
        """Setter pour celsius avec validation"""
        if valeur < -273.15:
            raise ValueError("Température en dessous du zéro absolu !")
        self._celsius = valeur
    
    @property
    def fahrenheit(self):
        """Conversion en Fahrenheit"""
        return self._celsius * 9/5 + 32
    
    @fahrenheit.setter
    def fahrenheit(self, valeur):
        """Définir la température en Fahrenheit"""
        self.celsius = (valeur - 32) * 5/9  # Utilise le setter celsius

temp = Temperature(25)
print(f"Température : {temp.celsius}°C = {temp.fahrenheit}°F")

# Modifier via celsius
temp.celsius = 0
print(f"Après modification : {temp.celsius}°C = {temp.fahrenheit}°F")

# Modifier via fahrenheit
temp.fahrenheit = 100
print(f"100°F = {temp.celsius}°C")

# Validation
try:
    temp.celsius = -300
except ValueError as e:
    print(f"\nErreur : {e}")

## 5. `@attribut.deleter` : contrôler la suppression

`@attribut.deleter` permet de définir le comportement lors de la suppression d'une propriété avec `del`.

In [ ]:
class Personne:
    def __init__(self, nom, email):
        self._nom = nom
        self._email = email
    
    @property
    def email(self):
        return self._email
    
    @email.setter
    def email(self, valeur):
        if "@" not in valeur:
            raise ValueError("Email invalide")
        self._email = valeur
    
    @email.deleter
    def email(self):
        print("Suppression de l'email")
        self._email = None

p = Personne("Alice", "alice@example.com")
print(f"Email : {p.email}")

# Modifier
p.email = "alice.dupont@example.com"
print(f"Nouveau email : {p.email}")

# Supprimer
del p.email
print(f"Email après suppression : {p.email}")

## 6. Pourquoi `@property` est préféré aux getters/setters

### Avantages de `@property`

1. **Syntaxe naturelle** : `obj.attribut` au lieu de `obj.get_attribut()`
2. **Refactoring facile** : on peut commencer avec un attribut public, puis ajouter de la logique plus tard
3. **Pythonic** : suit la philosophie "We're all consenting adults here"
4. **Lazy evaluation** : calcul à la demande
5. **Validation transparente** : l'utilisateur n'a pas besoin de savoir qu'il y a de la validation

In [ ]:
# Exemple de refactoring sans casser l'API

# Version 1 : attribut simple
class Cercle_V1:
    def __init__(self, rayon):
        self.rayon = rayon  # Public direct

c1 = Cercle_V1(5)
c1.rayon = 10  # Utilisation directe

# Version 2 : ajout de validation sans changer l'API
class Cercle_V2:
    def __init__(self, rayon):
        self._rayon = rayon
    
    @property
    def rayon(self):
        return self._rayon
    
    @rayon.setter
    def rayon(self, valeur):
        if valeur < 0:
            raise ValueError("Le rayon doit être positif")
        self._rayon = valeur
    
    @property
    def aire(self):
        import math
        return math.pi * self._rayon ** 2

c2 = Cercle_V2(5)
c2.rayon = 10  # Même syntaxe, mais avec validation !
print(f"Rayon : {c2.rayon}, Aire : {c2.aire:.2f}")

try:
    c2.rayon = -5
except ValueError as e:
    print(f"Erreur : {e}")

## 7. Validation avec `@property`

Les propriétés sont idéales pour valider les données lors de la modification.

In [ ]:
class Employe:
    def __init__(self, nom, age, salaire):
        self.nom = nom
        self.age = age  # Utilise le setter
        self.salaire = salaire  # Utilise le setter
    
    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self, valeur):
        if not isinstance(valeur, int):
            raise TypeError("L'âge doit être un entier")
        if not 18 <= valeur <= 70:
            raise ValueError("L'âge doit être entre 18 et 70 ans")
        self._age = valeur
    
    @property
    def salaire(self):
        return self._salaire
    
    @salaire.setter
    def salaire(self, valeur):
        if not isinstance(valeur, (int, float)):
            raise TypeError("Le salaire doit être un nombre")
        if valeur < 0:
            raise ValueError("Le salaire ne peut pas être négatif")
        self._salaire = valeur
    
    def __repr__(self):
        return f"Employe('{self.nom}', {self.age} ans, {self.salaire}€)"

# ✅ Création valide
emp = Employe("Alice", 30, 50000)
print(emp)

# ✅ Modification valide
emp.age = 35
emp.salaire = 55000
print(emp)

# ❌ Validations échouées
test_cases = [
    ("age", 15, "Âge < 18"),
    ("age", 80, "Âge > 70"),
    ("salaire", -1000, "Salaire négatif"),
]

for attr, valeur, description in test_cases:
    try:
        setattr(emp, attr, valeur)
    except (ValueError, TypeError) as e:
        print(f"❌ {description} : {e}")

## 8. Attributs en lecture seule

Pour créer un attribut en lecture seule, on définit seulement le getter, pas le setter.

In [ ]:
from datetime import datetime

class Commande:
    _compteur = 0
    
    def __init__(self, produits):
        Commande._compteur += 1
        self._id = Commande._compteur
        self._date_creation = datetime.now()
        self.produits = produits
    
    @property
    def id(self):
        """ID en lecture seule"""
        return self._id
    
    @property
    def date_creation(self):
        """Date de création en lecture seule"""
        return self._date_creation
    
    @property
    def total(self):
        """Total calculé dynamiquement"""
        return sum(p['prix'] * p['quantite'] for p in self.produits)
    
    def __repr__(self):
        return f"Commande #{self.id} - {self.total}€"

cmd1 = Commande([
    {"nom": "Laptop", "prix": 999.99, "quantite": 1},
    {"nom": "Souris", "prix": 29.99, "quantite": 2}
])

cmd2 = Commande([
    {"nom": "Clavier", "prix": 79.99, "quantite": 1}
])

# ✅ Lecture
print(f"Commande {cmd1.id} : {cmd1.total}€")
print(f"Commande {cmd2.id} : {cmd2.total}€")
print(f"Créée le : {cmd1.date_creation}")

# ❌ Tentative de modification
try:
    cmd1.id = 999
except AttributeError as e:
    print(f"\nErreur : {e}")

try:
    cmd1.total = 0
except AttributeError as e:
    print(f"Erreur : {e}")

## 9. `__slots__` : optimisation mémoire

`__slots__` permet de définir explicitement les attributs d'une classe, ce qui :
- **Réduit l'utilisation mémoire** (~40-50%)
- **Accélère l'accès aux attributs**
- **Empêche l'ajout dynamique** d'attributs

⚠️ À utiliser quand on crée beaucoup d'instances (milliers+)

In [ ]:
import sys

# Sans __slots__
class PointSansSlots:
    def __init__(self, x, y):
        self.x = x
        self.y = y

# Avec __slots__
class PointAvecSlots:
    __slots__ = ['x', 'y']  # Définir explicitement les attributs
    
    def __init__(self, x, y):
        self.x = x
        self.y = y

p1 = PointSansSlots(1, 2)
p2 = PointAvecSlots(1, 2)

print(f"Taille sans __slots__ : {sys.getsizeof(p1.__dict__)} bytes")
print(f"Taille avec __slots__ : {sys.getsizeof(p2)} bytes")

# Sans __slots__ : __dict__ existe
print(f"\n__dict__ de p1 : {p1.__dict__}")

# Avec __slots__ : pas de __dict__
try:
    print(p2.__dict__)
except AttributeError as e:
    print(f"p2 n'a pas de __dict__ : {e}")

# Ajout dynamique
p1.z = 3  # ✅ Possible sans __slots__
print(f"\np1.z = {p1.z}")

try:
    p2.z = 3  # ❌ Impossible avec __slots__
except AttributeError as e:
    print(f"Erreur avec __slots__ : {e}")

In [ ]:
# Démonstration de l'économie mémoire
import tracemalloc

def mesurer_memoire(classe, n=10000):
    tracemalloc.start()
    objets = [classe(i, i*2) for i in range(n)]
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    return peak / 1024  # En KB

mem_sans = mesurer_memoire(PointSansSlots)
mem_avec = mesurer_memoire(PointAvecSlots)

print(f"Mémoire pour 10,000 instances :")
print(f"Sans __slots__ : {mem_sans:.2f} KB")
print(f"Avec __slots__ : {mem_avec:.2f} KB")
print(f"Économie : {(1 - mem_avec/mem_sans) * 100:.1f}%")

### `__slots__` avec héritage

In [ ]:
class Point2D:
    __slots__ = ['x', 'y']
    
    def __init__(self, x, y):
        self.x = x
        self.y = y

class Point3D(Point2D):
    __slots__ = ['z']  # Seulement les nouveaux attributs
    
    def __init__(self, x, y, z):
        super().__init__(x, y)
        self.z = z

p3d = Point3D(1, 2, 3)
print(f"Point3D : ({p3d.x}, {p3d.y}, {p3d.z})")

# Vérifier les slots
print(f"Slots de Point2D : {Point2D.__slots__}")
print(f"Slots de Point3D : {Point3D.__slots__}")

## Pièges courants

### Piège 1 : `__` n'est pas vraiment privé

In [ ]:
# Ne pas utiliser __ pour la "sécurité"
class Securite:
    def __init__(self):
        self.__mot_de_passe = "secret123"  # Pas vraiment sécurisé !
    
    def verifier(self, mdp):
        return mdp == self.__mot_de_passe

s = Securite()

# ❌ Illusion de sécurité
print(f"Mot de passe 'caché' : {s._Securite__mot_de_passe}")

# Pour la vraie sécurité, utilisez des bibliothèques de cryptographie
# et ne stockez jamais de mots de passe en clair !

### Piège 2 : `@property` trop complexe

In [ ]:
# ❌ MAUVAIS : property avec effets de bord lourds
class MauvaiseAPI:
    def __init__(self):
        self._donnees = None
    
    @property
    def donnees(self):
        # ❌ Ne devrait pas faire de requête réseau dans un getter
        if self._donnees is None:
            import time
            print("Chargement des données... (simulation)")
            time.sleep(1)  # Simule une requête lente
            self._donnees = [1, 2, 3]
        return self._donnees

# L'utilisateur ne s'attend pas à une attente lors de l'accès
api = MauvaiseAPI()
print("Accès à api.donnees...")
print(api.donnees)  # Surprise : 1 seconde d'attente !

# ✅ BON : méthode explicite pour les opérations lourdes
class BonneAPI:
    def __init__(self):
        self._donnees = None
    
    def charger_donnees(self):  # ✅ Méthode explicite
        print("Chargement explicite...")
        self._donnees = [1, 2, 3]
    
    @property
    def donnees(self):
        if self._donnees is None:
            raise ValueError("Appelez charger_donnees() d'abord")
        return self._donnees

api2 = BonneAPI()
api2.charger_donnees()  # L'utilisateur sait que c'est long
print(api2.donnees)  # Accès rapide

### Piège 3 : Oublier le setter

In [ ]:
# ❌ Property sans setter (quand on en a besoin)
class Config:
    def __init__(self):
        self._debug = False
    
    @property
    def debug(self):
        return self._debug
    # ❌ Oubli du setter

config = Config()
print(f"Debug : {config.debug}")

try:
    config.debug = True  # ❌ Erreur
except AttributeError as e:
    print(f"Erreur : {e}")

# ✅ Avec setter
class ConfigCorrecte:
    def __init__(self):
        self._debug = False
    
    @property
    def debug(self):
        return self._debug
    
    @debug.setter
    def debug(self, valeur):
        self._debug = bool(valeur)

config2 = ConfigCorrecte()
config2.debug = True  # ✅
print(f"\nDebug : {config2.debug}")

## Mini-exercices

### Exercice 1 : Classe Temperature avec validation

Créez une classe `Temperature` avec :
- Attribut privé `_celsius`
- `@property celsius` avec validation (>= -273.15)
- `@property kelvin` pour conversion (K = C + 273.15)
- `@kelvin.setter` pour définir la température en Kelvin

Testez les conversions et la validation.

In [ ]:
# Votre code ici


### Solution Exercice 1

In [ ]:
class Temperature:
    ZERO_ABSOLU = -273.15
    
    def __init__(self, celsius=0):
        self.celsius = celsius  # Utilise le setter
    
    @property
    def celsius(self):
        return self._celsius
    
    @celsius.setter
    def celsius(self, valeur):
        if valeur < self.ZERO_ABSOLU:
            raise ValueError(
                f"Température {valeur}°C en dessous du zéro absolu ({self.ZERO_ABSOLU}°C)"
            )
        self._celsius = valeur
    
    @property
    def kelvin(self):
        return self._celsius - self.ZERO_ABSOLU
    
    @kelvin.setter
    def kelvin(self, valeur):
        self.celsius = valeur + self.ZERO_ABSOLU
    
    def __repr__(self):
        return f"Temperature({self.celsius}°C = {self.kelvin}K)"

# Tests
t = Temperature(25)
print(t)

t.celsius = 0
print(f"0°C = {t.kelvin}K")

t.kelvin = 0
print(f"0K = {t.celsius}°C")

try:
    t.celsius = -300
except ValueError as e:
    print(f"\nErreur : {e}")

### Exercice 2 : Classe Email avec validation

Créez une classe `Email` avec :
- `@property adresse` avec validation (doit contenir @ et .)
- `@property utilisateur` (partie avant @) - lecture seule
- `@property domaine` (partie après @) - lecture seule
- Méthode `est_valide()` qui vérifie la validité

Testez avec différentes adresses.

In [ ]:
# Votre code ici


### Solution Exercice 2

In [ ]:
class Email:
    def __init__(self, adresse):
        self.adresse = adresse  # Utilise le setter pour validation
    
    @property
    def adresse(self):
        return self._adresse
    
    @adresse.setter
    def adresse(self, valeur):
        if not self._valider_format(valeur):
            raise ValueError(f"Adresse email invalide : {valeur}")
        self._adresse = valeur
    
    @staticmethod
    def _valider_format(adresse):
        """Validation basique"""
        if "@" not in adresse:
            return False
        parties = adresse.split("@")
        if len(parties) != 2:
            return False
        utilisateur, domaine = parties
        if not utilisateur or not domaine:
            return False
        if "." not in domaine:
            return False
        return True
    
    @property
    def utilisateur(self):
        """Partie avant @ (lecture seule)"""
        return self._adresse.split("@")[0]
    
    @property
    def domaine(self):
        """Partie après @ (lecture seule)"""
        return self._adresse.split("@")[1]
    
    def est_valide(self):
        return self._valider_format(self._adresse)
    
    def __repr__(self):
        return f"Email('{self.adresse}')"

# Tests valides
emails_valides = [
    "alice@example.com",
    "bob.martin@company.fr",
    "contact@sub.domain.org"
]

for addr in emails_valides:
    e = Email(addr)
    print(f"✅ {e.adresse}")
    print(f"   Utilisateur: {e.utilisateur}, Domaine: {e.domaine}\n")

# Tests invalides
emails_invalides = [
    "invalid",
    "@example.com",
    "user@",
    "user@@domain.com",
    "user@domain"
]

for addr in emails_invalides:
    try:
        Email(addr)
    except ValueError as e:
        print(f"❌ {e}")

### Exercice 3 : Point avec `__slots__`

Créez deux classes `Point2D` (sans `__slots__`) et `Point2DOptimise` (avec `__slots__`) avec :
- Attributs `x`, `y`
- Méthode `distance_origine()` qui calcule la distance à l'origine

Comparez l'utilisation mémoire pour 10 000 instances.

In [ ]:
# Votre code ici


### Solution Exercice 3

In [ ]:
import math
import tracemalloc

class Point2D:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def distance_origine(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    def __repr__(self):
        return f"Point2D({self.x}, {self.y})"

class Point2DOptimise:
    __slots__ = ['x', 'y']
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def distance_origine(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    def __repr__(self):
        return f"Point2DOptimise({self.x}, {self.y})"

def mesurer_memoire_et_temps(classe, n=10000):
    import time
    
    # Mémoire
    tracemalloc.start()
    debut = time.time()
    
    points = [classe(i, i*2) for i in range(n)]
    
    fin = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
    return peak / 1024, (fin - debut) * 1000  # KB, ms

# Comparaison
mem_normal, temps_normal = mesurer_memoire_et_temps(Point2D)
mem_optimise, temps_optimise = mesurer_memoire_et_temps(Point2DOptimise)

print("Comparaison pour 10,000 instances :")
print(f"\nSans __slots__ :")
print(f"  Mémoire : {mem_normal:.2f} KB")
print(f"  Temps : {temps_normal:.2f} ms")

print(f"\nAvec __slots__ :")
print(f"  Mémoire : {mem_optimise:.2f} KB")
print(f"  Temps : {temps_optimise:.2f} ms")

print(f"\nGains :")
print(f"  Mémoire : {(1 - mem_optimise/mem_normal) * 100:.1f}% d'économie")
print(f"  Temps : {(1 - temps_optimise/temps_normal) * 100:.1f}% plus rapide")

# Test fonctionnel
p1 = Point2D(3, 4)
p2 = Point2DOptimise(3, 4)

print(f"\nTest fonctionnel :")
print(f"{p1} - Distance: {p1.distance_origine():.2f}")
print(f"{p2} - Distance: {p2.distance_origine():.2f}")

## Résumé

Dans ce notebook, vous avez appris :

- ✅ Convention `_` pour les attributs/méthodes protégés (par convention)
- ✅ Name mangling `__` pour éviter les collisions dans l'héritage
- ✅ `@property` pour des getters Pythonic
- ✅ `@attribut.setter` pour des setters avec validation
- ✅ `@attribut.deleter` pour contrôler la suppression
- ✅ Préférer `@property` aux getters/setters Java-style
- ✅ Validation transparente avec les propriétés
- ✅ Attributs en lecture seule (property sans setter)
- ✅ `__slots__` pour optimiser la mémoire
- ✅ Éviter les pièges : `__` n'est pas privé, property trop complexe, oublier le setter

**Prochaine étape** : Héritage simple et multiple (`super()`, MRO, mixins)